# Data Download #

This notebook includes the instructions and runs the code to download the data for the workshop.

## GATAs ChIP ##

### Data from Yan et al. ###

The original paper is avaiable here: https://www.cell.com/cell/fulltext/S0092-8674(13)00942-2?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS0092867413009422%3Fshowall%3Dtrue .
Accession number is [GSE49402](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE49402).

We download different GATA datasets.

Downloaded files needs to be processed, because originally they include summary from MACS2 and columns are not ordered the way HOMER accepts. In addition, the provied regions are 1-based, instead of 0-based. For this a specific R script is used.

In [ ]:
%%bash

echo "; Creating data directory"

data_dir="peak_sets"
mkdir -p $data_dir

echo "; Downloading data to a directory."

# GATA1
wget -qO - https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1208nnn/GSM1208749/suppl/GSM1208749_batch2_chrom1_LoVo_GATA1_PassedQC_peaks.txt.gz | gunzip > GATA1_LoVo_peaks.txt
mv GATA1_LoVo_peaks.txt $data_dir/GATA1_LoVo_peaks.txt ;

# GATA2 (failed QC according to the authors)
wget -qO - https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1208nnn/GSM1208619/suppl/GSM1208619_batch1_chrom1_LoVo_GATA2_FailedQC_peaks.txt.gz | gunzip > GATA2_LoVo_peaks.txt
mv GATA2_LoVo_peaks.txt $data_dir/GATA2_LoVo_peaks.txt ;

# GATA4 
wget -qO - https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1208nnn/GSM1208620/suppl/GSM1208620_batch1_chrom1_LoVo_GATA4_PassedQC_peaks.txt.gz | gunzip > GATA4_LoVo_peaks.txt
mv GATA4_LoVo_peaks.txt $data_dir/GATA4_LoVo_peaks.txt ;

The generation of the narrowPeak files in this case is done from the following files:

* ```peak_sets/GATA1_LoVo_peaks.txt```
* ```peak_sets/GATA2_LoVo_peaks.txt```
* ```peak_sets/GATA4_LoVo_peaks.txt```

These files start with a header that needs to be skipped. Once done, we reformat the file to follow the ```.narrowPeak``` format, add the "chr" string in the first column, and skip non-canonical chromosomes. The resulting file is then randomly subset to select 5000 random peaks. 

In [8]:
%%bash

data_dir="peak_sets"

taipale_peaks=($(find $data_dir -name "GATA*_peaks.txt"))
nb_of_sampled_peaks=2000

for dataset in ${taipale_peaks[@]}; do

    # Format the peaks to narrowPeak with summit in column 10
    # We also remove non-canonical chromosomes
    awk -v OFS='\t' '$0 !~ "#" && $0 !~ "chr" {print}' $dataset \
    | tail -n +2 \
    | awk -v OFS='\t' '{print "chr"$1, $2, $3, ".", ".", ".", ".", ".", ".", $5}' \
    | grep -v "chrNT" \
    > ${dataset//".txt"/".narrowPeak"}
    
    peak_file=${dataset//".txt"/".narrowPeak"}
    
    # Subset n peaks
    [ $(wc -l < $peak_file) -ge $nb_of_sampled_peaks ] && shuf -n $nb_of_sampled_peaks $peak_file \
    > ${peak_file//"_peaks.narrowPeak"/"_"$nb_of_sampled_peaks"peaks.narrowPeak"} || cat $peak_file > ${peak_file//"_peaks.narrowPeak"/"_"$nb_of_sampled_peaks"peaks.narrowPeak"}
    
done

The peak sets in `.narrowPeak` format then needs to be converted into fasta sequences.

In [7]:
%%bash

## Data and output directories:
data_dir="peak_sets" ;
output_dir="fasta_peak_sets" ;
genomes_dir="/home/ievarau/Documents/genomes"
nb_of_sampled_peaks=2000

## Creating output directory:
mkdir -p $output_dir ;

## hg18 datasets
hg18datasets=($(find $data_dir -name "GATA*_"${nb_of_sampled_peaks}"peaks.narrowPeak"))

for dataset in ${hg18datasets[@]}
do
    dset_bname=$(basename ${dataset%"_"${nb_of_sampled_peaks}"peaks.narrowPeak"});
    
    echo $dset_bname
    
    awk -v OFS='\t' '{print $1, $2 + $10, $2 + $10 + 1}' $dataset \
    | bedtools slop -g ${genomes_dir}/hg18/hg18.chrom.sizes -b 50 \
    | bedtools getfasta -fi ${genomes_dir}/hg18/hg18.fa -bed stdin \
    > ${output_dir}/${dset_bname}_fasta_sequences.fa ;

done ;

peak_sets/GATA4_LoVo_2000peaks.narrowPeak


### Data from UniBind ###

Below are the peak datasets that were processed for UniBind 2021 and we now will re-process:

* ```NR3C1_A549-lung-carcinoma-0h_peaks.narrowPeak```
* ```NR3C1_A549-lung-carcinoma-10min_peaks.narrowPeak```
* ```NR3C1_A549-lung-carcinoma-8h_peaks.narrowPeak```

Due to the large number of peaks in some of the files, which then prolongs the motif discovery, we sample a specific number of peaks, when files have more peaks than that number.


In [21]:
%%bash

## Subsetting random 5000 peaks

echo "; Sampling peaks to reduce the runtime of analysis."

data_dir="peak_sets"
nb_of_sampled_peaks=40000

for peak_file in $data_dir/NR3C1*.narrowPeak ; do

    echo $peak_file

    [ $(wc -l < $peak_file) -ge $nb_of_sampled_peaks ] && shuf -n $nb_of_sampled_peaks $peak_file \
    > ${peak_file//"_peaks.narrowPeak"/"_"${nb_of_sampled_peaks}"peaks.narrowPeak"} || cat $peak_file > ${peak_file//"_peaks.narrowPeak"/"_"${nb_of_sampled_peaks}"peaks.narrowPeak"}

done ;


; Sampling peaks to reduce the runtime of analysis.
peak_sets/NR3C1_A549-lung-carcinoma-0h_peaks.narrowPeak
peak_sets/NR3C1_A549-lung-carcinoma-10min_peaks.narrowPeak
peak_sets/NR3C1_A549-lung-carcinoma-8h_peaks.narrowPeak


In [22]:
%%bash

## Data and output directories:
data_dir="peak_sets" ;
output_dir="fasta_peak_sets" ;
genomes_dir="/home/ievarau/Documents/genomes"
nb_of_sampled_peaks=40000

## Creating output directory:
mkdir -p $output_dir ;

## hg38 datasets:
hg38datasets=($(find $data_dir -name "NR3C1*_"${nb_of_sampled_peaks}"peaks.narrowPeak"))

for dataset in ${hg38datasets[@]}
do
    dset_bname=$(basename ${dataset%"_"${nb_of_sampled_peaks}"peaks.narrowPeak"});
    
    awk -v OFS='\t' '{print $1, $2 + $10, $2 + $10 + 1}' $dataset \
    | bedtools slop -g ${genomes_dir}/hg38/hg38.chrom.sizes -b 50 \
    | bedtools getfasta -fi ${genomes_dir}/hg38/hg38.fa -bed stdin \
    > ${output_dir}/${dset_bname}_fasta_sequences.fa ;

done ;